# Speadsheet calculator
### This notebook will extract data and format it into an excel sheet. Run the cells in order (omit cells if necessary)
### Jeffrey Zhou
### Fall 2022

In [1]:
import pymrio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This following cell will parse data from the EXIO3 database into workable matrices and dictonaries. BEFORE RUNNING, note that you will need to download the ZIP versions of IOT data from the EXIO3 website. Include the full pathname to the ZIP file in the 'path' argument in the <parse_exiobase3> method call

In [66]:
'''
Data setup
Note: this will take a while to run. Something like 3 mins
'''

exio3 = pymrio.parse_exiobase3(path='/Users/jeffreyzhou/Desktop/MRIO/Data/IOT_2019_pxp.zip')
all_regions = exio3.get_regions()
all_sectors = exio3.get_sectors()

#include all missing calculations with calc_all()
exio3.calc_all()

#US Z matrix - transaction matrix
us_z = exio3.Z.aggregate('US')

#US Y matrix - final demand matrix
us_y = exio3.Y.aggregate('US')

trade_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    trade_matrix_dict[region] = us_z.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200

demand_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    demand_matrix_dict[region] = us_y.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200

trade_calcvalues_dict = {}
    
for region in all_regions:
    tm = trade_matrix_dict[region]
    d = {}
    for sector in all_sectors:
        d[sector] = tm[sector].values.sum()
    in_series = pd.Series(data = d, index = all_sectors)
    trade_calcvalues_dict[region] = in_series

US_total_trade = {}

for sector in all_sectors:
    in_dict = {}
    for region in all_regions:
        if region == 'US': # comment out this line if including domestic inputs
            continue
        in_dict[region] = trade_calcvalues_dict[region][sector]
    US_total_trade[sector] = in_dict

This next cell will create the database which we will append columns of data onto, and eventually export as a spreadsheet. 

In [ ]:
# Data could be = all_sectors or all_regions. can add new axis as needed
df = pd.DataFrame(data=all_regions)


In [4]:
# Save function to add columns to the spreadsheet

def save(database, lst, label):
    database[label] = lst
    return database

In [29]:
#function that plots graphs of specific width and height.

def plot_func(x, y, xlabel, ylabel, title):
    fig = plt.figure(figsize=(16, 5))
    ax = fig.add_subplot(111)
    ax.bar(x,y)
    fig.suptitle(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=18)
    plt.ylabel(ylabel, fontsize=16)

    
    plt.show()

## To add more columns to the spreadsheet, just extract an array of length == all_sectors/all_regions (depending on Y axis) and call the save() function

In [47]:
# Consumption impacts (% of consumption that come from imports)
imports = exio3.impacts.D_imp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)']
consumption = exio3.impacts.D_cba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)']
    
    
imports = imports.to_dict()
consumption = consumption.to_dict()

impacts_percent= {}
count = 0
for sector in all_sectors:
    d = consumption[sector]
    n = imports[sector]
    if d == 0:
        count += 1
        if n != 0:
            print("ERROR")
        impacts_percent[sector] = 'NA'
        continue
    percent = n / (d + n)
    impacts_percent[sector] = percent
    
x, y = zip(*impacts_percent.items())
df = save(df, y, "Consumption impacts (% of consumption GHG that come from imports)")

In [48]:
# Total GHG from imports
y2 = exio3.impacts.D_imp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y2, "Total GHG from imports")

In [49]:
# Total GHG from exports
y3 = exio3.impacts.D_exp.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y3, "Total GHG from exports")

In [46]:
# Total GHG from consumption (Imports and domestic)
y4 = exio3.impacts.D_cba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df,y4, "Total GHG from consumption (Imports and domestic)")

In [50]:
# Total GHG from production (Domestic)
y5 = exio3.impacts.D_pba.US.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, y5, "Total GHG from production (Domestic)")

In [10]:
# Total GHG produced by Region R-axis
regional_prod_ghg = exio3.impacts.D_pba_reg.loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
df = save(df, regional_prod_ghg, "Total GHG produced by Region")

In [41]:
# Total GHG produced by Region per sector

ghgpersector = {}
for sector in all_sectors:
    ghgpersector[sector] = []

for region in all_regions:
    region_list = exio3.impacts.F[region].loc['GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'].to_list()
    for i in range(len(all_sectors)):
        ghgpersector[all_sectors[i]].append(region_list[i])


for sector in all_sectors:
    df = save(df, ghgpersector[sector], sector)


In [ ]:
# Trade Imports into the US (excluding Domestic trade)

trade_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    trade_matrix_dict[region] = us_z.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200

demand_matrix_dict = {}
count = 0
lower_bound = 0
upper_bound = 200

for region in all_regions:
    demand_matrix_dict[region] = us_y.iloc[lower_bound:upper_bound]
    lower_bound += 200
    upper_bound += 200
    
US_total_trade = {}

for sector in all_sectors:
    in_dict = {}
    for region in all_regions:
        if region == 'US':
            continue
        in_dict[region] = trade_calcvalues_dict[region][sector]
    US_total_trade[sector] = in_dict


for sector in all_sectors:
    x, y = zip(*US_total_trade[sector].items())
    df = save(df, y, sector)

In [ ]:
# Run to check numbers in database before exporting spreadsheet
df

## You're done! Run the next cell to export the spreadsheet. Change the name_of_file to desired location

In [65]:
# Run this cell to print and export the data in 'DF' into a spreadsheet

name_of_file = 'Sources of imports into US 2019' # Change this string to desired filename
df.to_excel(f'{name_of_file}.xlsx')